In [15]:
IN_COLAB = 'google.colab' in str(get_ipython())
TRAIN = True

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/Github/Abstract-generator/bumbleBERT/notebooks
    batch_size = 10 #32
    eval_batch_size = 10 #32
    epochs = 10  # The number of epochs
else:
    batch_size = 128       # 3
    val_batch_size = 128  # 3
    epochs = 200  # The number of epochs

print('Running in colab:', IN_COLAB)
print('Training mode:', TRAIN)

Running in colab: False
Training mode: True


In [3]:
!pip install feedparser tokenizers transformers

In [4]:
import os, torch, time, math, sys, re, csv
import numpy as np

sys.path.append('..' + os.sep )
from src import default

from src.data import download as dl, tokenization as tkn, custom_dataset as cd

from torch.utils.data import DataLoader
from src.model.transformer_hf import TransformerModel
from src.model.batching import CustomBatch
from src.model.generate_text import gen_some_text
#from src.model.train_evaluate import train, evaluate
#from src.model.transformer import make_gpt_model # imports don't work

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Parameters

In [5]:
# ARCHITECTURE
maxLen     = 35 # maximum sentence length
vocabSize  = None # None if you want to let tokenizer do its thing
emsize     = 512 # embedding dimension
nhid       = 2048 # the dimension of the feedforward network model in torch.nn.TransformerEncoder
nlayers    = 12 # the number of torch.nn.TransformerEncoderLayer in torch.nn.TransformerEncoder
nhead      = 8 # the number of heads in the multiheadattention models
dropout    = 0.2 # the dropout value

# TOKENIZER
tknzerType = 'BPE' # type of tokenizing algorithm
trainTokenizer = False # whether to train a new tokenizer or use one already trained

### Download Dataset

In [6]:
# download data
nbrResults = 10**3 # number of data samples to download
filename = f'arxiv_{nbrResults}'
extension = '.csv'
filepath = default.RAW_DATA_DIR + os.sep + filename + extension

if not os.path.exists(filepath):
    dl.arxiv_api( default.RAW_DATA_DIR, filepath, max_results=nbrResults ) # TODO : CHANGE SO THAT NOT CONSTANTLY LOADING DATA
print(f'>> Using {filename} for training <<')

>> Using arxiv_1000 for training <<


### Format Dataset

Uses a custom dataset class, which is an iterable and callable structure that returns a sample from our dataset. Within this custom dataset, can determine all preprocessing.

In [7]:
# create dataset
dataset = cd.ArxivDataset(filepath)

### Training Tokenizer

Training of a custom tokenizer. Many options possible here, check the tokenizer training functions to try out various strategies. If he tokenizer for the dataset has already been trained, no need to run this again.

In [8]:
trainTokenizer: _ = tkn.train_custom_tokenizer(tknzerType, dataset, filename
                                            , default.TOK_DIR
                                            , vocabSize
                                            , **default.special_token_lst)

### Loading Tokenizer and Splitting Datasets

For some reason, torch tokenizers are not callable as trained. This is confusing, but c'est la vie! Instead, need to load it from file it was saved in using the PreTrainedTokenizerFast class (__call__) implemented in here. Once that's done, you can add this tokenizer as a transform to your dataset! Useful.

We also split the dataset here into training, testing and validation datasets.

In [9]:
tknzrFile = default.TOK_DIR + os.sep + filename + '_' + tknzerType + '.json'

# load PreTrainedTokenizerFast, for __call__. __call__ not implemented in
# the base Tokenizer class... that sounds silly, but it is what it is
tknzr = tkn.load_tokenizer(tknzrFile, **default.special_token_lst)

if vocabSize is None: vocabSize = tknzr.vocab_size

# set tknzr as the transform
dataset.set_transform( tknzr )

# separate dataset into train, test valid TODO : make into a function
fracTrain, fracTest, fracVal = ( 0.7, 0.2, 0.1)
trainTestVal = [ np.floor(fracTrain*len(dataset))\
                    , np.floor(fracTest*len(dataset))\
                    , len(dataset) - ( np.floor( fracTrain*len(dataset) ) +
                    np.floor( fracTest*len(dataset) ) )
                    ]

trainDataset, testDataset, valDataset =\
        torch.utils.data.random_split(dataset, [int(x) for x in trainTestVal]
                                , generator=torch.Generator().manual_seed(42) )


### Creating DataLoaders

Training is done on batches, so we need a way to extract groupings of the data in the appropriate format for our transformer model.
Note that for transformers which we are training, dataloaders outputs both src (x[:-1] and tgt ([1:]).
The collation of batches for different transformer models we have vary. For HuggingFace it's ( maxLen x batch_size ) whereas I think that the Annotated Transformer has ( batch_size x maxLen ).

NOTE : Do not use the tokenizer before the training if you use num_workers>0!
FastTokenizer does not play nicely with forking if you use it before the forking of your data:
https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning

In [10]:
# create dataloaders
# uses collate function to transform batch to correct dimensions
def collate_wrapper(batch):
    return CustomBatch(batch, dim=0, maxLenModel=maxLen, padValue=tknzr.get_vocab()["<pad>"])

# dataloader for training
trainDataLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )
# dataloader for validation
valDataLoader = DataLoader(valDataset, batch_size=val_batch_size, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )

### Selecting model

Here we choose which model we shall use for training. For now, I've selected the black box Transformer from HuggingFace because the collate_fn I've written gives the correct input size force it... however this can easily be changed! 

In [11]:
# transformer from huggingface
# TODO : Change to the Annotated Transformer if I want
model = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
# learning rate Matt used with Adam is 0.5
paramsAdam  = [{'params' : model.parameters(), 'lr' : 0.5, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsAdamW = [{'params' : model.parameters(), 'lr' : 5e-5, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsSGD   = [{'params' : model.parameters(), 'lr' : 0.5, 'momentum' : 0.0, 'dampening' : 0.0, 'weight_decay' : 0.0}]

#optimizer = torch.optim.SGD( paramsSGD )
optimizer = torch.optim.Adam( paramsAdam )
#optimizer = torch.optim.AdamW( paramsAdamW )

# scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

### Training

Training loop!

In [12]:
def train( model, maxLen, dataLoader, device, vocabSize, epoch, optimizer_, scheduler_, criterion_ ):
    """
    Training loop that takes batches from dataLoader and pushes them to device
    to train. Will check if they're the same size of maxLen: if shorter, will
    reduces to longest length in batch. then trains according to optimizer,
    criterion and schedule.

    Input
        model (instance)        : model that is being trained
        maxLen (int)            : maximum sentence length
        dataLoader (instance)   : dataloader that batches data into tensors
        optimizer (instance)    : Not sure what type optimizers are
        criterion               :
        device (str)            : gpu or cpu
    Output
        None
    """

    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(maxLen).to(device)
    for i, batch in enumerate(dataLoader):
        #print((batch.src).is_pinned())
        src = (batch.src).to(device); tgt = (batch.tgt).to(device)

        optimizer_.zero_grad()
        if src.size(0) != maxLen:
            src_mask = model.generate_square_subsequent_mask(src.size(0)).to(device)

        output = model(src, src_mask)
        loss = criterion_(output.view(-1, vocabSize), tgt.reshape(-1))
        loss.backward()
        torch.torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer_.step()

        total_loss += loss.item()
        log_interval = 10
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(dataLoader),
                            scheduler_.get_last_lr()[0],
                            elapsed * 1000 / log_interval,
                            cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


# evaluation function outside of training - same as hugging face
def evaluate(eval_model, maxLen, dataLoader, nbrSamples, device, vocabSize, criterion_):
    """
    Takes a trained model, puts it in evaluation mode to see how well it
    performs on another set of data.

    Input
        eval_model (instance)   : model to be evaluated
        maxLen (int)            : maximum length possible/trained on
        dataLoader (instance)   : dataloader of the dataset that is evaluate on
        nbrSamples (int)        : Supposed to be number of samples, not sure I need
    Output
        loss of evaluated set
    """
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = eval_model.generate_square_subsequent_mask(maxLen).to(device)
    with torch.no_grad():
        for batch in dataLoader:
            src = (batch.src).to(device); tgt = (batch.tgt).to(device)
            print(batch.src)
            if src.size(0) != maxLen:
                src_mask = eval_model.generate_square_subsequent_mask(
                                                    src.size(0)).to(device)
            output = eval_model(src, src_mask)
            output_flat = output.view(-1, vocabSize)
            #total_loss += len(src) * criterion_(output_flat
            total_loss += criterion_(output_flat
                                                , tgt.reshape(-1)).item()
    #print('data_source: ',nbrSamples)
    return total_loss / (nbrSamples - 1) # nbrSamples -x-> len(dataLoader)


In [13]:
best_val_loss = float("inf")
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train( model, maxLen, trainDataLoader, device, vocabSize, epoch, optimizer, scheduler, criterion)
    val_loss = evaluate(model, maxLen, valDataLoader, len(valDataset), device, vocabSize, criterion)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
                                     # Why is math.exp so large????
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

    
# save best model (two methods)
modelFull = default.MODEL_DIR + os.sep + f'{filename}_epoch{epochs}.pth'
modelWeights = default.MODEL_DIR + os.sep + f'{filename}_weights_epoch{epochs}.pth'
# approach 1: save model (class) entirely (uses pickle)
torch.save(model, modelFull)
# approach 2: save model weights
torch.save(model.state_dict(), modelWeights)

KeyboardInterrupt: 

### Text Generation

Here I've simply taken the code Matt uses to generate text.

In [ ]:
if not TRAIN:
    customFilename = 'arxiv_10000'
    customEpochs = 10
    modelFull = default.MODEL_DIR + os.sep + f'{customFilename}_epoch{customEpochs}.pth'
    modelWeights = default.MODEL_DIR + os.sep + f'{customFilename}_weights_epoch{customEpochs}.pth'
    
    # approach 1: load model (class) entirely (uses pickle)
    modelFullLoad = torch.load(modelFullPath, map_location=device)

    # approach 2: load model weights, need to have some parameter or something 
    modelLoad = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)
    modelWeightsLoad = modelLoad.load_state_dict( torch.load(modelWeightsPath) )

In [ ]:
# inspect both models
#print('model_A info...\n', modelFullLoad)
#print('\nmodel_B info...\n', modelWeightsLoad)

#print('model_A == model_B:', modelFullLoad == modelWeightsLoad)
#model = modelFullLoad
# Text generation example
prompt = 'Electrons are protons.'
ngen = 500
decode_style = 'sample_full'
model.to('cpu')
generated_text = gen_some_text(
    model, tknzr, 'cpu', maxLen, text_prompt=prompt, tokens_to_gen=ngen, vis=False,
    decode_style=decode_style)
print("Text prompt:\n", prompt)
print("Number of tokens to generate:", ngen)
print("Generated_text:\n", generated_text)

# TODO: alternative generation
# currently 'greedy method'
# see: https://huggingface.co/blog/how-to-generate